In [ ]:
import numpy as np

from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from validation_model import FaceValidateV1
from settings import *

In [ ]:
model = FaceValidateV1.load_state_dict(VALIDATION_MODEL_PATH)

class EmbeddedFacesDataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        